In [8]:
from collections import Counter, defaultdict
import jsonlines
import json
import numpy as np
import pandas as pd
import re

In [10]:
def clean_text(text):
    return text.replace('\xa0', ' ').strip()

articles_filename = './data/tyzhden_articles.jl'

articles = []
with jsonlines.open(articles_filename) as reader:
    for obj in reader:
        obj['title'] = clean_text(obj['title'])
        obj['subtitle'] = clean_text(obj['subtitle'])
        if 'зібрав найважливіші новини' in obj['subtitle']:
            continue
        obj['text'] = clean_text(obj['text'])
        if obj['text']:
            articles.append(obj)
        
orig_df = pd.DataFrame(articles)
orig_df

,link,title,posted_at,subtitle,text,source_url,source
0,https://tyzhden.ua/News/242033,Позачергове засідання Ради відбудеться 30 бере...,202003292250,"Арахамія зазначив, що завтра може відбутись од...","В 29 березня, 30 березня, відбудеться позачерг...",http://tyzhden.ua/,Тиждень.ua
1,https://tyzhden.ua/News/242025,"Бойовики на Донбасі 10 разів порушили режим ""т...",202003281943,Поранення отримали 2 українських військових,З початку 28 березня проросійські бойовики 10 ...,,
2,https://tyzhden.ua/News/242024,У Чернівецькій області зафіксовано майже 60 ви...,202003281817,"Зазначається, що троє пацієнтів, у яких вчора ...",У Чернівецькій області станом на 28 березня за...,http://tyzhden.ua/,Тиждень.ua
3,https://tyzhden.ua/News/242023,"Трепак заявив, що справа Гандзюк вийшла на фін...",202003281622,"За його словами, перерозподіл повноважень між ...","Заступник генпрокурора Віктор Трепак заявив, щ...",http://tyzhden.ua/,Тиждень.ua
4,https://tyzhden.ua/News/242026,На Донбасі внаслідок обстрілів двоє військових...,202003291138,Окупанти 15 разів порушили режим тиші.,"На Донбасі протягом минулої доби, 28 березня, ...",http://tyzhden.ua/,Тиждень.ua
...,...,...,...,...,...,...,...
5080,https://tyzhden.ua/News/235191,Червоний Хрест відправив понад дві тонни гумдо...,201909121006,Місцевим жителям доставили лікарські препарати...,Міжнародний комітет Червоного Хреста направив ...,http://tyzhden.ua/,Тиждень.ua
5081,https://tyzhden.ua/News/235190,Заступник голови Нацполіції Аброськін спростув...,201909121001,Наразі заступник голови НПУ перебуває у Маріуп...,Генерал поліції другого рангу В'ячеслав Абрось...,http://tyzhden.ua/,Тиждень.ua
5082,https://tyzhden.ua/News/235189,ВР сьогодні розгляне законопроект про відстроч...,201909120948,"Крім того, нардепи розглянуть законопроекти пр...","Верховна рада України у четвер, 12 вересня, ро...",http://tyzhden.ua/,Тиждень.ua
5083,https://tyzhden.ua/News/235188,"Стало відомо, коли Зеленський проведе зустріч ...",201909120927,Зустріч відбудеться 12 вересня.,Президент України Володимир Зеленський проведе...,http://tyzhden.ua/,Тиждень.ua


In [6]:
# Для розмітки даних
df = orig_df.copy()

for i, row in df.iterrows():
    text_splitted = row['text'].split('Як повідомлялося, ')
    if len(text_splitted) > 1:
        df.loc[i, 'text'] = text_splitted[0]
        df.loc[i, 'before'] = text_splitted[1]
        continue

    text_splitted = row['text'].split('Нагадаємо, ')
    if len(text_splitted) > 1:
        df.loc[i, 'text'] = text_splitted[0]
        df.loc[i, 'before'] = text_splitted[1]
        continue

    text_splitted = row['text'].split('Зауважимо: ')
    if len(text_splitted) > 1:
        df.loc[i, 'text'] = text_splitted[0]
        df.loc[i, 'before'] = text_splitted[1]
        continue

    df.loc[i, 'before'] = ''

df

,link,title,posted_at,subtitle,text,source_url,source,before
0,https://tyzhden.ua/News/242033,Позачергове засідання Ради відбудеться 30 бере...,202003292250,"Арахамія зазначив, що завтра може відбутись од...","В 29 березня, 30 березня, відбудеться позачерг...",http://tyzhden.ua/,Тиждень.ua,
1,https://tyzhden.ua/News/242025,"Бойовики на Донбасі 10 разів порушили режим ""т...",202003281943,Поранення отримали 2 українських військових,З початку 28 березня проросійські бойовики 10 ...,,,
2,https://tyzhden.ua/News/242024,У Чернівецькій області зафіксовано майже 60 ви...,202003281817,"Зазначається, що троє пацієнтів, у яких вчора ...",У Чернівецькій області станом на 28 березня за...,http://tyzhden.ua/,Тиждень.ua,Covid-19 спалахнув на ринку морепродуктів в ки...
3,https://tyzhden.ua/News/242023,"Трепак заявив, що справа Гандзюк вийшла на фін...",202003281622,"За його словами, перерозподіл повноважень між ...","Заступник генпрокурора Віктор Трепак заявив, щ...",http://tyzhden.ua/,Тиждень.ua,генеральний прокурор Ірина Венедіктова 27 бере...
4,https://tyzhden.ua/News/242026,На Донбасі внаслідок обстрілів двоє військових...,202003291138,Окупанти 15 разів порушили режим тиші.,"На Донбасі протягом минулої доби, 28 березня, ...",http://tyzhden.ua/,Тиждень.ua,
...,...,...,...,...,...,...,...,...
5139,https://tyzhden.ua/News/235191,Червоний Хрест відправив понад дві тонни гумдо...,201909121006,Місцевим жителям доставили лікарські препарати...,Міжнародний комітет Червоного Хреста направив ...,http://tyzhden.ua/,Тиждень.ua,8 серпня Міжнародний комітет Червоного хреста ...
5140,https://tyzhden.ua/News/235190,Заступник голови Нацполіції Аброськін спростув...,201909121001,Наразі заступник голови НПУ перебуває у Маріуп...,Генерал поліції другого рангу В'ячеслав Абрось...,http://tyzhden.ua/,Тиждень.ua,троє заступників голови Національної поліції С...
5141,https://tyzhden.ua/News/235189,ВР сьогодні розгляне законопроект про відстроч...,201909120948,"Крім того, нардепи розглянуть законопроекти пр...","Верховна рада України у четвер, 12 вересня, ро...",http://tyzhden.ua/,Тиждень.ua,13 лютого уряд митне оформлення автомобілів на...
5142,https://tyzhden.ua/News/235188,"Стало відомо, коли Зеленський проведе зустріч ...",201909120927,Зустріч відбудеться 12 вересня.,Президент України Володимир Зеленський проведе...,http://tyzhden.ua/,Тиждень.ua,


In [7]:
df.loc[df.before != '']

,link,title,posted_at,subtitle,text,source_url,source,before
2,https://tyzhden.ua/News/242024,У Чернівецькій області зафіксовано майже 60 ви...,202003281817,"Зазначається, що троє пацієнтів, у яких вчора ...",У Чернівецькій області станом на 28 березня за...,http://tyzhden.ua/,Тиждень.ua,Covid-19 спалахнув на ринку морепродуктів в ки...
3,https://tyzhden.ua/News/242023,"Трепак заявив, що справа Гандзюк вийшла на фін...",202003281622,"За його словами, перерозподіл повноважень між ...","Заступник генпрокурора Віктор Трепак заявив, щ...",http://tyzhden.ua/,Тиждень.ua,генеральний прокурор Ірина Венедіктова 27 бере...
11,https://tyzhden.ua/News/242020,Зеленський призначив нового начальника Генерал...,202003281418,"Відповідно до тексту документу, Корнійчук приз...",Президент Володимир Зеленський призначив генер...,http://tyzhden.ua/,Тиждень.ua,Сергія Наєва командувачем Об'єднаних сил Зброй...
12,https://tyzhden.ua/News/242018,Командувачем Об'єднаних сил ЗСУ призначено Наєва,202003281338,Раніше Сергій Наєв вже очолював ООС.,Сергія Наєва призначено командувачем Об'єднани...,,,16 березня 2018 року президент Петро Порошенко...
19,https://tyzhden.ua/News/242010,Північна Македонія стала членом НАТО,202003271929,Прапор Північної Македонії 30 березня піднімут...,"Північна Македонія сьогодні, 27 березня, офіці...",http://tyzhden.ua/,Тиждень.ua,"що 19 березня, генсек НАТО Єнс Столтенберг у р..."
...,...,...,...,...,...,...,...,...
5137,https://tyzhden.ua/News/235193,США запевнили Зеленського у підтримці на шляху...,201909121029,"Зокрема, дипломати закликали Росію виконати вс...",У посольстві Сполучених Штатів Америки в Украї...,http://tyzhden.ua/,Тиждень.ua,що 9 вересня США в ОБСЄ Росію звільнити решту ...
5138,https://tyzhden.ua/News/235192,"На Луганщині поліцейські затримали бойовика ""ЛНР""",201909121024,"За інформацією правоохоронців, чоловік прямува...",Поліція у Луганській області затримала колишнь...,http://tyzhden.ua/,Тиждень.ua,у Луганській області правоохоронці бойовика мо...
5139,https://tyzhden.ua/News/235191,Червоний Хрест відправив понад дві тонни гумдо...,201909121006,Місцевим жителям доставили лікарські препарати...,Міжнародний комітет Червоного Хреста направив ...,http://tyzhden.ua/,Тиждень.ua,8 серпня Міжнародний комітет Червоного хреста ...
5140,https://tyzhden.ua/News/235190,Заступник голови Нацполіції Аброськін спростув...,201909121001,Наразі заступник голови НПУ перебуває у Маріуп...,Генерал поліції другого рангу В'ячеслав Абрось...,http://tyzhden.ua/,Тиждень.ua,троє заступників голови Національної поліції С...


In [59]:
filename_base = './data/some_articles_{}.jsonl'

filename_no = 1
file_batches = defaultdict(list)
i = 0
for _, row in df.iterrows():
    link = row['link'].strip()
    title = row['title'].strip()
    subtitle = row['subtitle'].strip()
    text = row['text'].strip()
    if text:
        row_data = {
            'text': link + ' . ' + title + ' . ' + subtitle + ' . ' + text.replace('\n', ' ')
        }
        file_batches[filename_no].append(row_data)
        if i < 299:
            i += 1
        else:
            i = 0
            filename_no += 1

for i, rows in file_batches.items():
    with jsonlines.open(filename_base.format(i), mode='w') as writer:
        for row in rows:
            writer.write(row)